**Import various modules and packages for image processing, machine learning, and deep learning, including TensorFlow, Keras, scikit-learn, PyTorch, and Hugging Face Transformers, preparing for image classification and model training.**

In [ ]:
!pip install pydot graphviz
sudo apt-get install graphviz

In [ ]:
import os
from shutil import copy2
from collections import Counter
import random
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from PIL import Image
from sklearn.model_selection import train_test_split
import shutil
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Rescaling
from tensorflow.keras import layers, regularizers
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

#For Cross Validation 
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#For basic Keras model training
from tensorflow import keras
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, Dropout

#For State-of-art pre-trained models
from tensorflow.keras import models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50, ResNet152V2, ResNet101
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

from tensorflow.keras.applications import EfficientNetB0, EfficientNetB7, EfficientNetB6, EfficientNetV2S, EfficientNetV2M
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import MobileNetV2

#For ArcFace
import math
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input, Softmax
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from IPython.display import Image

**Check and remove corrupted images**

In [ ]:
def delete_corrupted_images(directory):
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                continue

            file_path = os.path.join(root, filename)

            try:
                with Image.open(file_path) as img:
                    img.verify()
            except (IOError, SyntaxError) as e:
                print(f"PIL cannot process {file_path}, deleting: {e}")
                os.remove(file_path)
                continue  

            try:
                image_bytes = tf.io.read_file(file_path)
                image = tf.io.decode_image(image_bytes, channels=3, expand_animations=False)
                image = tf.image.resize(image, [256, 256])
            except tf.errors.InvalidArgumentError as e:
                print(f"TensorFlow cannot process {file_path}, deleting: {e}")
                os.remove(file_path)

        for dirname in dirs:
            dir_path = os.path.join(root, dirname)
            if not os.listdir(dir_path):  
                print(f"Removing empty directory: {dir_path}")
                os.rmdir(dir_path)

directory = 'train_images'
delete_corrupted_images(directory)

**Data Understanding and Overview**

In [ ]:
dataset_path = 'train_images'

def analyze_images(dataset_path):
    hotel_image_counts = {}

    for hotel_id in os.listdir(dataset_path):
        hotel_dir = os.path.join(dataset_path, hotel_id)
        
        if os.path.isdir(hotel_dir):
            image_count = len([name for name in os.listdir(hotel_dir) if os.path.isfile(os.path.join(hotel_dir, name))])
            hotel_image_counts[hotel_id] = image_count

    total_hotels = len(hotel_image_counts)
    min_images = min(hotel_image_counts.values())
    max_images = max(hotel_image_counts.values())
    avg_images = sum(hotel_image_counts.values()) / total_hotels

    image_count_frequency = Counter(hotel_image_counts.values())

    top_10_common_counts = image_count_frequency.most_common(10)
    counts = [count for count, num_hotels in top_10_common_counts]
    hotels = [num_hotels for count, num_hotels in top_10_common_counts]

    plt.figure(figsize=(10, 6))
    plt.bar(range(len(counts)), hotels, tick_label=counts)
    plt.xlabel('Number of Images per Hotel')
    plt.ylabel('Number of Hotels')
    plt.title('Top 10 Most Common Image Counts')
    plt.show()

    
    print(f"Total number of hotels: {total_hotels}")
    print(f"Minimum number of images in a hotel: {min_images}")
    print(f"Maximum number of images in a hotel: {max_images}")
    print(f"Average number of images per hotel: {avg_images:.2f}")
    print("The top 10 most common image counts are:")
    for count, num_hotels in top_10_common_counts:
        print(f"{num_hotels} hotels have {count} images")

analyze_images(dataset_path)

**Challenge 1 - Class Diversity:** The dataset comprises a **large number of classes (1,678 distinct hotels)**, which presents a substantial challenge in terms of learning distinct features for each class.

**Challenge 2 - Class Imbalance:** There is **significant imbalance** in the dataset, with some hotels represented by a single image while others have over a thousand. This disparity can lead to overfitting and poor generalization for underrepresented classes.

**Challenge 3 - Insufficient Learning Samples:** Most hotels have **fewer than 15 images** available, which may not provide enough data for the model to effectively learn the variance within each class.

**Challenge 4 - Subtle Variations:** Hotel rooms often share similar aesthetics and structures, much like nuances in human faces, making it **difficult to distinguish between different hotels**.

**Challenge 5 - Occlusions from Privacy Masks:** The presence of **occlusion masks** (red masks) adds complexity to the task, as they can **cover significant portions** of the images, obscuring important details that are necessary for accurate classification.

**Challenge 6 - Potential Overfitting:** With a limited number of images per class, there is a risk of models overfitting to the training data, which could result in poor performance on unseen data.

**Challenge 7 - Background Noise and Variability:** The variability in lighting, decor, and camera angles within hotel room images introduces additional complexity, potentially leading to background noise that can confuse the model.

**The process_images function resizes images, applies random red masks to them, and saves the processed images to a specified directory**

In [ ]:
#A function to resize images
def resize_image (image_path, target_size=(256,256)):
    """Resize an image to the target size."""
    image = load_img(image_path)
    image = image.resize(target_size)
    return image

#A funtion to apply masks
def apply_mask (image, mask):
    """Apply a red mask to an image."""
    image_array = img_to_array (image)
    mask_array = img_to_array (mask)
    #Maks's red color is the pure red color [255,0,0]
    red_mask = (mask_array[:,:,0] == 255) & (mask_array[:,:,1] == 0) & (mask_array[:,:,2] == 0)
    #Apply the red mask to each channel of the image
    image_array[red_mask] = [255,0,0] #This sets the color to red wherever the mask is present
    return array_to_img(image_array)

#A function to process the images by resizing and applying masks
def process_images(train_images_dir, train_masks_dir, output_dir, target_size=(256, 256)):
    """Process images by resizing and applying masks."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Initialize a counter for processed images
    processed_count = 0

    # List all subdirectories in train_images_dir
    subdirs = [d for d in os.listdir(train_images_dir) if os.path.isdir(os.path.join(train_images_dir, d))]
    
    mask_files = [f for f in os.listdir(train_masks_dir) if f.lower().endswith('.png')]

    for subdir in subdirs:
        subdir_path = os.path.join(train_images_dir, subdir)
        image_files = [f for f in os.listdir(subdir_path) if f.lower().endswith('.jpg')]

        for image_file in image_files:
            try:
                image_path = os.path.join(subdir_path, image_file)
                mask_file = random.choice(mask_files)  # Random mask for each image
                mask_path = os.path.join(train_masks_dir, mask_file)

                resized_image = resize_image(image_path, target_size)
                resized_mask = resize_image(mask_path, target_size)

                masked_image = apply_mask(resized_image, resized_mask)

                output_path = os.path.join(output_dir, f"{subdir}_{image_file}")
                masked_image.save(output_path)
                processed_count += 1
            except Exception as e:
                print (f"Error processing image {image_path}: {e}")

    print(f"Processed {processed_count} images and saved to {output_dir}")

def copy_files(files, source_dir, dest_dir):
    for f in files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(dest_dir, f))

train_images_dir = 'train_images'
train_masks_dir = 'train_masks'
processed_images_dir ='processed_images'

if os.path.exists(processed_images_dir):
    shutil.rmtree(processed_images_dir)
os.makedirs(processed_images_dir)

process_images(train_images_dir, train_masks_dir, processed_images_dir)

**Create training and validation datasets from the 'train_images' directory, using TensorFlow to preprocess images into batches of 48 with a size of 256x256 pixels; Split the data into 70% for training and 30% for validation and testing**

In [ ]:
directory = 'processed_images'

full_train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=directory,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=48,
    image_size=(256,256),
    shuffle=True,
    seed=123,
    validation_split=0.3, 
    subset='training', 
    interpolation='bilinear', 
    follow_links=False,
    crop_to_aspect_ratio=False
)

full_validation_test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=48,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.3,  
    subset='validation',  
    interpolation='bilinear',
    crop_to_aspect_ratio=False
)

**Calculate the total number of batches in a validation and test dataset, split it evenly into two subsets for validation and testing**

In [ ]:
total_batches = len(full_validation_test_dataset)
batches_per_subset = total_batches // 2

full_validation_dataset = full_validation_test_dataset.take(batches_per_subset)

full_test_dataset = full_validation_test_dataset.skip(batches_per_subset)

**Define a data augmentation pipeline using Keras, incorporating random flips, rotations, zooms, contrast and brightness adjustments, resizing to 280x280 pixels, cropping back to 256x256 pixels, and translations to enhance the diversity of the training dataset**

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical", input_shape=(256, 256, 3)),  
        layers.RandomRotation(0.2),  
        layers.RandomZoom(0.2),  
        layers.RandomContrast(factor=0.1),  
        layers.RandomBrightness(factor=0.1),  
        layers.Resizing(280, 280),  
        layers.RandomCrop(height=256, width=256),  
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1), 
    ]
)


**Part II: Basic Keras Models**

**Build a machine learning workflow for image classification using TensorFlow and Keras:**

**1.** **Set up essential parameters**: class names, the number of classes, and the number of epochs for training. The dataset is prepared using the ImageDataGenerator for image rescaling, ensuring proper input to the neural network;
   
**2.** A versatile model creation function is defined to **construct CNN models with optional data augmentation and dropout**, making it adaptable to various scenarios. The compile_and_train_model function takes care of compiling and training the models;

**3.** **Visualize training results** by the plot_training_history function, providing plots of training and validation accuracy and loss, aiding in the interpretation of the model's performance over epochs.
   
**4.** **Implements K-Fold cross-validation** to ensure the model's robustness and generalizability. It generates training and validation data generators for each fold, trains the model, and computes the average accuracy, offering a comprehensive evaluation of the model's performance.

In [ ]:
class_names = full_train_dataset.class_names
num_classes = len(class_names)
epochs = 10
directory = 'full_train_dataset'
n_splits = 3

In [ ]:
def create_model(num_classes, use_data_augmentation=False, dropout_rate=None):
    layers_list = []
    if use_data_augmentation:
        layers_list.append(data_augmentation)
    layers_list.extend([
        layers.Rescaling(1./255, input_shape=(256, 256, 3)),
        layers.Conv2D(16, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
    ])
    if dropout_rate is not None:
        layers_list.append(layers.Dropout(dropout_rate))
    layers_list.extend([
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return Sequential(layers_list)

def compile_and_train_model(model, train_ds, val_ds, epochs):
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )
    return model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs
    )

def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1,2,2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show
    

def cross_validate_model(create_model_func, model_params, epochs, n_splits):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=123)

    fold_no = 1
    scores = []

    for train_index, val_index in kfold.split(np.arange(total_batches)):
        print(f'Fold {fold_no} --')

        train_subset = tf.data.experimental.cardinality(full_train_dataset).numpy()
        val_subset = tf.data.experimental.cardinality(full_validation_dataset).numpy()

        train_batches = int(train_subset * len(train_index) / total_batches)
        val_batches = int(val_subset * len(val_index) / total_batches)

        train_ds = full_train_dataset.take(train_batches)
        val_ds = full_validation_dataset.skip(val_batches).take(val_batches)

        model = create_model_func(**model_params)
        compile_and_train_model(model, train_ds, val_ds, epochs)

        scores.append(model.evaluate(val_ds))

        fold_no += 1

    scores = np.array(scores)
    print('Scores for each fold:', scores)
    print('Average score:', np.mean(scores, axis=0))
    print('Standard deviation:', np.std(scores, axis=0))

**A basic Keras convolutional neural network model is created without any data augmentation or dropout features**

In [ ]:
#Basic Keras model without data augmentation and dropout
model_basic = create_model(num_classes)
history_basic = compile_and_train_model(model_basic, full_train_dataset, full_validation_dataset, epochs)
plot_training_history(history_basic)

model_basic_config = {"func": create_model, "params": {"num_classes": num_classes}}
print("Running cross-validation for the basic Keras model")
average_accuracy_basic = cross_validate_model(model_basic_config['func'], model_basic_config['params'], epochs, n_splits)
print(f"Average accuracy for basic model: {average_accuracy_basic}")

**Incorporating data augmentation to the Keras convolutional neural network model**

In [ ]:
# Basic Keras Model with data augmentation
model_with_aug = create_model(num_classes, use_data_augmentation=True)
history_with_aug = compile_and_train_model(model_with_aug, full_train_dataset, full_validation_dataset, epochs)
plot_training_history(history_with_aug)

model_with_augmentation_config = {"func": create_model, "params": {"num_classes": num_classes, "use_data_augmentation": True}}
print("\nRunning cross-validation for the basic Keras model with augmentation")
average_accuracy_augmentation = cross_validate_model(model_with_augmentation_config['func'], model_with_augmentation_config['params'], epochs, n_splits)
print(f"Average accuracy for model with augmentation: {average_accuracy_augmentation}")

**Incorporating data augmentation and dropout to the Keras convolutional neural network model**

In [ ]:
# Basic Keras Model with data augmentation and dropout
model_dropout_aug = create_model(num_classes, use_data_augmentation=True, dropout_rate=0.2)
history_dropout_aug = compile_and_train_model(model_dropout_aug, full_train_dataset, full_validation_dataset, epochs)
plot_training_history(history_dropout_aug)

model_with_augmen_dropout_config = {"func": create_model, "params": {"num_classes": num_classes, "use_data_augmentation": True, "dropout_rate": 0.2}}
print("\nRunning cross-validation for the basic Keras model with augmentation and dropout")
average_accuracy_dropout = cross_validate_model(model_with_augmen_dropout_config['func'], model_with_augmen_dropout_config['params'], epochs, n_splits)
print(f"Average accuracy for model with augmentation and dropout: {average_accuracy_dropout}")

**Part III: More advanced and pre-trained convolutional neural network models**

**The limitations of a basic Keras model** in handling such a complex and nuanced task stem from its typically shallow architecture, which might lack the capacity to capture the subtle differences between highly similar hotel rooms. 

While **VGG and Inception**, the deeper convolutional neural networks, offer improvements by delving deeper into the image structure through its multiple layers.

Given the extensive size of our dataset, comprising **22,244 images across 1,674 classes**, coupled with the constraints of **limited computational resources**, we opted for a strategic approach. We **selected a smaller subset of the dataset (the top 500 classes with the most images)** to conduct preliminary tests on various deeper CNN pre-trained models. By concentrating on **tweaking the top-performing models**, we plan to gradually make them even better. This careful way of doing things helps us deal with our limited resources and still get the best results we can.

In [ ]:
full_dataset_directory = "processed_images"

image_counts = {}

for class_dir in os.listdir(full_dataset_directory):
    class_path = os.path.join(full_dataset_directory, class_dir)
    if os.path.isdir(class_path):
        image_counts[class_dir] = len([entry for entry in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, entry))])

top_500_classes = Counter(image_counts).most_common(500)

subset_directory = "subset_top_500_hotels"  
os.makedirs(subset_directory, exist_ok=True)

for class_dir, _ in top_500_classes:
    original_class_path = os.path.join(full_dataset_directory, class_dir)
    subset_class_path = os.path.join(subset_directory, class_dir)
    os.makedirs(subset_class_path, exist_ok=True)
    
    for image in os.listdir(original_class_path):
        src_image_path = os.path.join(original_class_path, image)
        dst_image_path = os.path.join(subset_class_path, image)
        copy2(src_image_path, dst_image_path)

print("The subset with the top 500 classes has been created.")

In [ ]:
dataset_path = 'subset_top_500_hotels'
analyze_images(dataset_path)

In [ ]:
subset_directory = 'subset_top_500_hotels'
delete_corrupted_images(subset_directory)

subset_train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=subset_directory,
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=48,
    image_size=(256,256),
    shuffle=True,
    seed=123,
    validation_split=0.3, 
    subset='training', 
    interpolation='bilinear', 
    follow_links=False,
    crop_to_aspect_ratio=False
)

subset_validation_test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=subset_directory,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=48,
    image_size=(256, 256),
    shuffle=True,
    seed=123,
    validation_split=0.3,  
    subset='validation',  
    interpolation='bilinear',
    crop_to_aspect_ratio=False
)

total_batches = len(subset_validation_test_dataset)
batches_per_subset = total_batches // 2
subset_validation_dataset = subset_validation_test_dataset.take(batches_per_subset)
subset_test_dataset = subset_validation_test_dataset.skip(batches_per_subset)

In [ ]:
def prepare_model(model_name, num_classes, input_shape=(256, 256, 3), learning_rate=0.001):
    if model_name == 'VGG16':
        base_model = VGG16(include_top=False, input_shape=input_shape, weights='imagenet')
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(include_top=False, input_shape=input_shape, weights='imagenet')
    elif model_name == 'ResNet152V2':
        base_model = ResNet152V2(include_top=False, input_shape=input_shape, weights='imagenet')
    elif model_name == 'ResNet101':
        base_model = ResNet101(include_top=False, input_shape=input_shape, weights='imagenet')
    elif model_name == 'ResNet50':
        base_model = ResNet50(include_top=False, input_shape=input_shape, weights='imagenet')
    elif model_name == 'DenseNet121':
        base_model = DenseNet121(include_top=False, input_shape=input_shape, weights='imagenet')
    elif model_name == 'EfficientNetB7':
        base_model = EfficientNetB7(include_top=False, input_shape=input_shape, weights='imagenet')
    elif model_name == 'EfficientNetB6':
        base_model = EfficientNetB6(include_top=False, input_shape=input_shape, weights='imagenet')
    elif model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')
    elif model_name == 'EfficientNetV2S':
        base_model = EfficientNetV2S(include_top=False, input_shape=input_shape, weights='imagenet')
    elif model_name == 'EfficientNetV2M':
        base_model = EfficientNetV2M(include_top=False, input_shape=input_shape, weights='imagenet')
    else:
        raise ValueError(f"Unsupported model: {model_name}")

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)  
    x = Dense(1024, activation='relu')(x)  
    output = Dense(num_classes, activation='softmax')(x)  

    model = Model(inputs=base_model.input, outputs=output)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def train_model(model, train_dataset, validation_dataset, epochs=10):
    history = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=10,
        verbose=1
    )
    return history

**VGG-16:** Progress from a basic Keras model to the more advanced and pre-trained VGG-16 model for image classification. **VGG-16 is a powerful convolutional neural network pre-trained on the ImageNet dataset**, we are hoping to leverage deeper and more complex architectures that have been proven effective on a wide range of image recognition tasks.

In [ ]:
class_names = subset_train_dataset.class_names
num_classes = len(class_names)

In [ ]:
model_name = 'VGG16'  
model_VGG16 = prepare_model(model_name, num_classes)
history = train_model(model_VGG16, subset_train_dataset, subset_validation_dataset)

In [ ]:
model_name = 'InceptionV3'  
model_InceptionV3 = prepare_model(model_name, num_classes)
history = train_model(model_InceptionV3, subset_train_dataset, subset_validation_dataset)

**VGG and Inception** represent earlier convolutional neural network (CNN) designs focusing on **depth** and **multi-scale** processing, while models like **ResNet, DenseNet, and EfficientNet** are more advanced iterations that incorporate additional mechanisms to handle the challenges of deeper network structures. 

**ResNet** introduces a novel approach to facilitate training deeper networks through skip connections;

**DenseNet** streamlines the training of deep architectures by densely connecting each layer to every other layer, ensuring maximum information flow between layers. This design not only enhances performance with a more efficient parameter usage but also helps alleviate the vanishing-gradient problem, making deep networks easier to train;

**EfficientNet** optimizes CNN scaling by uniformly increasing depth, width, and resolution with fixed coefficients, leading to state-of-the-art performance on image classification tasks. 

In [ ]:
model_name = 'ResNet152V2'  
model_ResNet152V2 = prepare_model(model_name, num_classes)
history = train_model(model_ResNet152V2, subset_train_dataset, subset_validation_dataset)

In [ ]:
model_name = 'ResNet101'  
model_ResNet101 = prepare_model(model_name, num_classes)
history = train_model(model_ResNet101, subset_train_dataset, subset_validation_dataset)

In [ ]:
model_name = 'ResNet50'  
model_ResNet50 = prepare_model(model_name, num_classes)
history = train_model(model_ResNet50, subset_train_dataset, subset_validation_dataset)

In [ ]:
model_name = 'DenseNet121'  
model_DenseNet121 = prepare_model(model_name, num_classes)
history = train_model(model_DenseNet121, subset_train_dataset, subset_validation_dataset)

In [ ]:
model_name = 'EfficientNetB7'  
model_EfficientNetB7 = prepare_model(model_name, num_classes)
history = train_model(model_EfficientNetB7, subset_train_dataset, subset_validation_dataset)

In [ ]:
model_name = 'EfficientNetB6'  
model_EfficientNetB6 = prepare_model(model_name, num_classes)
history = train_model(model_EfficientNetB6, subset_train_dataset, subset_validation_dataset)

In [ ]:
model_name = 'EfficientNetB0'  
model_EfficientNetB0 = prepare_model(model_name, num_classes)
history = train_model(model_EfficientNetB0, subset_train_dataset, subset_validation_dataset)

In [ ]:
model_name = 'EfficientNetV2S'  
model_EfficientNetV2S = prepare_model(model_name, num_classes)
history = train_model(model_EfficientNetV2S, subset_train_dataset, subset_validation_dataset)

In [ ]:
model_name = 'EfficientNetV2M'  
model_EfficientNetV2M = prepare_model(model_name, num_classes)
history = train_model(model_EfficientNetV2M, subset_train_dataset, subset_validation_dataset)

**Among all the pre-trained models, EfficientNetB0 stood out; we continue to use the EfficientNetB0 architecture as a base, leveraging transfer learning, regularization, and data augmentation to enhance performance on potentially complex datasets.**

In [ ]:
def build_model(num_classes, input_shape=(256, 256, 3), l1_factor=0, l2_factor=0, dropout_rate=0, use_data_augmentation=False):
    base_model = EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')
    base_model.trainable = False
    
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    
    if use_data_augmentation:
        data_augmentation = tf.keras.Sequential([
            layers.RandomFlip("horizontal_and_vertical"),  
            layers.RandomRotation(0.2),  
            layers.RandomZoom(0.2),  
            layers.RandomContrast(factor=0.1),  
            layers.RandomBrightness(factor=0.1),  
            layers.Resizing(280, 280),  
            layers.RandomCrop(height=256, width=256),  
            layers.RandomTranslation(height_factor=0.1, width_factor=0.1), 
        ])
        x = data_augmentation(x)
    
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    if dropout_rate > 0:
        x = tf.keras.layers.Dropout(dropout_rate)(x)
    
    x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=l1_factor, l2=l2_factor))(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l1_l2(l1=l1_factor, l2=l2_factor))(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

def compile_and_fit(model, train_dataset, validation_dataset, epochs=50):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_dataset, validation_data=validation_dataset, epochs=epochs, verbose=1)
    return history

In [ ]:
#EfficientNetB0 Model without augmentation, drop out or regularization
Efficient_base_model = build_model(num_classes)
history = compile_and_fit(Efficient_base_model, subset_train_dataset, subset_validation_dataset)

In [ ]:
#EfficientNetB0 Model with l1+l2 Regularization, dropout and augmentation
model_with_reg = build_model(num_classes, l1_factor=1e-5, l2_factor=1e-4, dropout_rate=0.2, use_data_augmentation=True)
history_with_reg = compile_and_fit(model_with_reg, subset_train_dataset, subset_validation_dataset)

In [ ]:
#EfficientNetB0 Model with l1+l2 Regularization
model_with_reg = build_model(num_classes, l1_factor=1e-5, l2_factor=1e-4)
history_with_reg = compile_and_fit(model_with_reg, subset_train_dataset, subset_validation_dataset)

In [ ]:
#EfficientNetB0 Model with dropout (0.2)
model_with_reg = build_model(num_classes, dropout_rate=0.2)
history_with_reg = compile_and_fit(model_with_reg, subset_train_dataset, subset_validation_dataset)

In [ ]:
#EfficientNetB0 Model with augmentation
model_with_reg = build_model(num_classes, use_data_augmentation=True)
history_with_reg = compile_and_fit(model_with_reg, subset_train_dataset, subset_validation_dataset)

In [ ]:
#EfficientNetB0 Model with l1+l2 Regularization and dropout 
model_with_reg = build_model(num_classes, l1_factor=1e-5, l2_factor=1e-4, dropout_rate=0.2)
history_with_reg = compile_and_fit(model_with_reg, subset_train_dataset, subset_validation_dataset)

### **ArcFace Layer**

To address the challenges akin to those in facial recognition, incorporating **ArcFace** into our model presents a compelling solution. The ArcMarginProduct class, inspired by the ArcFace method, is a custom TensorFlow layer designed to **significantly enhance the discriminative power of feature embeddings** in deep learning models, particularly beneficial for classification tasks where subtle differences between classes, such as facial features or hotel rooms, are paramount. ArcFace stands out for its innovative approach of enforcing an angular margin between classes in the feature space. This technique effectively amplifies the model's **sensitivity to intra-class variations** and ensures a clearer separation between different classes.

<img width="1300" alt="Screenshot 2024-02-10 at 5 47 44 PM" src="https://github.com/Yiyi-Luo/Capstone-Project-Hotel-Detection-to-Combat-Human-Trafficking/assets/149438809/347415ed-0fea-43fa-b38f-6f777098c0f7">

By adjusting the cosine distance between feature vectors and class centers with a **scale (s) and margin (m)** parameter, ArcFace makes the model more adept at distinguishing between closely related categories. The ArcMarginProduct layer implements this by applying the angular margin in the cosine space, offering options for easy margin settings and **label smoothing (ls_eps)** to promote training stability and enhance generalization. This layer is particularly useful in scenarios demanding high discriminative capabilities, such as identifying nuanced differences in hotel room categories or in facial recognition tasks, ensuring the model learns robust and distinct features for each class, thereby improving accuracy and generalization in scenarios characterized by subtle variances.

<img width="1176" alt="Screenshot 2024-02-10 at 5 48 22 PM" src="https://github.com/Yiyi-Luo/Capstone-Project-Hotel-Detection-to-Combat-Human-Trafficking/assets/149438809/c4559c51-fc88-44e3-9126-676c95ded963">

#### **References:**
##### 1. https://arxiv.org/pdf/1801.07698.pdf (**Authors:** Jiankang Deng, Jia Guo, Jing Yang, Niannan Xue, Irene Kotsia, and Stefanos Zafeiriou)
##### 2. [How to train with ArcFace loss to improve model classification accuracy | by Yiwen Lai](https://yiwenlai.medium.com/how-to-train-with-arcface-loss-to-improve-model-classification-accuracy-d4035195aeb9) (**Author:** Yiwen Lai)
##### 3. https://github.com/Niellai/ObjectDetection/blob/master/10_COVID19_ArcFace.ipynb?source=post_page-----d4035195aeb9-------------------------------- (**Author:** Niel Lai)
##### 4. https://www.kaggle.com/code/hidehisaarai1213/glret21-efficientnetb0-baseline-inference/notebook (**Author:** Hidehisa Arai)
##### 5. https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/blob/master/src/modeling/metric_learning.py (**Author:** Lyakaap)

**Preparing dataset for ArcFace**

In [ ]:
def prepare_for_arcface(images, labels):
    return (images, labels), labels

subset_train_dataset = subset_train_dataset.map(prepare_for_arcface)
subset_validation_dataset = subset_validation_dataset.map(prepare_for_arcface)
subset_test_dataset = subset_test_dataset.map(prepare_for_arcface)

In [ ]:
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf (Authors: Jiankang Deng, Jia Guo, Jing Yang, Niannan Xue, Irene Kotsia, and Stefanos Zafeiriou)
        https://www.kaggle.com/code/hidehisaarai1213/glret21-efficientnetb0-baseline-inference/notebook (Author: Hidehisa Arai)
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/ 
            blob/master/src/modeling/metric_learning.py (Author: Lyakaap)
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

**We freeze all but the last three layers of EfficientNetB0 for feature extraction, apply global average pooling, and optionally include dropout for regularization. The ArcMarginProduct layer, which requires both feature and label inputs, is utilized to enforce an angular margin that enhances the separability between classes.**

In [ ]:
def build_model_with_arcface(num_classes, input_shape=(256, 256, 3), dropout_rate=0, l1_factor=0, l2_factor=0):
    '''
    Build model using EfficientNetB0 as base and ArcFace as output layer.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf(Authors: Jiankang Deng, Jia Guo, Jing Yang, Niannan Xue, Irene Kotsia, and Stefanos Zafeiriou)
        How to train with ArcFace loss to improve model classification accuracy | by Yiwen Lai (Author: Yiwen Lai)
        https://github.com/Niellai/ObjectDetection/blob/master/10_COVID19_ArcFace.ipynb?source=post_page-----d4035195aeb9--------------------------------(Author: Niel Lai)
        https://www.kaggle.com/code/hidehisaarai1213/glret21-efficientnetb0-baseline-inference/notebook (Author: Hidehisa Arai)

    '''
    premodel = EfficientNetB0(include_top=False, input_shape=input_shape, weights='imagenet')
    
    # Freeze the layers except the last 3
    for layer in premodel.layers[:-3]:
        layer.trainable = False
    
    # Define the input layers
    inputs = tf.keras.Input(shape=input_shape)
    label_input = Input(shape=(num_classes,))  # Additional input for labels
    
    # Forward pass through the base model
    x = premodel(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    
    if dropout_rate > 0:
        x = Dropout(dropout_rate)(x)
    
    x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=l1_factor, l2=l2_factor))(x)
    
    output = ArcMarginProduct(n_classes=num_classes, s=30.0, m=0.5)([x, label_input])
    
    model = Model([inputs, label_input], output)
    
    return model

def compile_and_fit(model, train_dataset, validation_dataset, epochs=30, callbacks=None):
    
    model.compile(
        optimizer=tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-5),  
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    history = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=epochs,
        callbacks=callbacks,  
        verbose=1,
    )
    return history

In [ ]:
checkpoint_dir = 'model_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
model_path = os.path.join(checkpoint_dir, 'best_model.h5')


checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, mode='min')

callbacks = [
    ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, mode='min'),
]

input_shape = (256, 256, 3)
num_classes = 500  
dropout_rate = 0.2
l1_factor = 1e-4
l2_factor = 1e-4
model = build_model_with_arcface(num_classes, input_shape, l1_factor, l2_factor, dropout_rate)
model.summary()

In [ ]:
plot_model(model, to_file='model_architecture.png', show_shapes=True)
Image(filename='model_architecture.png')

In [ ]:
history = compile_and_fit(model, subset_train_dataset, subset_validation_dataset, epochs=30, callbacks=callbacks)

test_loss, test_accuracy = model.evaluate(subset_test_dataset)
print(f'Test accuracy: {test_accuracy}, Test loss: {test_loss}')